main_dev_pmgus.ipynb
# Pre-Market Momentum "Gap-up" Screener & Strategy (#1-DT, #2-Swing)

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import pytz  # timezone
import warnings
from datetime import datetime, timedelta
from tqdm import tqdm  # Visualize loop progress
from sklearn.linear_model import LinearRegression
from tenacity import retry, stop_after_attempt, wait_fixed

pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [24]:
import os
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv

# LEVEL #0
| INITIAL DATA IMPORT |
|-|
| LEVEL #0 |

In [25]:
# base file path and file path setup
base_dir_path = '/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/' 
std_file_name_str = 'tv_screen_gap-up_'

#####---------------------#####
# set the screen date
screen_date = '2024-11-26' # **** # SMTC
screen_date = '2024-11-27'

#####---------------------#####

file_type = '.csv'
filename = base_dir_path + std_file_name_str + screen_date + file_type

# read the csv file
print(filename)
trading_view_df = pd.read_csv(filename)
print(len(trading_view_df))
# print first 5 ticker Symbols only
print(trading_view_df['Symbol'].head(5))

/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/tv_screen_gap-up_2024-11-27.csv
1729
0     GOOG
1    GOOGL
2     META
3     TSLA
4    BRK.B
Name: Symbol, dtype: object


In [26]:
print(trading_view_df.columns)
print(len(trading_view_df))
trading_view_df.head(3)

Index(['Symbol', 'Description', 'Industry', 'Sector', 'Exchange', 'Index',
       'Market capitalization', 'Market capitalization - Currency', 'Price',
       'Price - Currency', 'Pre-market Open', 'Pre-market Open - Currency',
       'Pre-market Change', 'Pre-market Change - Currency',
       'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding',
       'Volume 1 day', 'Volume 1 week', 'Pre-market Volume',
       'Average Volume 10 days', 'Average Volume 30 days',
       'Average Volume 90 days', 'Volatility 1 day', 'Volatility 1 week',
       'Volatility 1 month', 'Volume Weighted Average Price 1 day',
       'Price to earnings ratio', 'Relative Volume at Time',
       'Relative Volume 1 day', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years',
       'Relative Volume 1 minute', 'Relative Volume 5 minutes',
       'Relative Volume 15 minutes', 'Relative Volume 30 minutes',
       'Relative Volume 1 hour', 'Relative Volume 2 hours',
       'Relative Volume 4 hours', 'Relati

,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Relative Volume at Time,Relative Volume 1 day,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 week,Relative Volume 1 month,High 1 month,High 1 month - Currency,High 3 months,High 3 months - Currency,High 6 months,High 6 months - Currency,High 52 weeks,High 52 weeks - Currency,High All Time,High All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month.1,High 1 month - Currency.1,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes
0,GOOG,Alphabet Inc.,Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500 Communication Services, S&P 100, Nasdaq US Large Cap Growth, Russell 3000, NASDAQ-100 Technology Sector, Russell 1000, NASDAQ Computer, S&P 500 ESG",2.078499e+12,USD,170.62,USD,170.54,USD,0.36,USD,0.210995,-0.046888,5.138646e+09,14937416,36333130.0,35195,20312170.7,1.909483e+07,1.835263e+07,1.218792,2.591361,2.235971,170.515,22.633150,0.796513,0.738561,1.025681,0.973451,1.039125,4.475506,6.824742,6.632638,3.729061,1.627667,0.716800,0.513050,0.410427,0.815222,184.025,USD,184.025,USD,193.31,USD,193.31,USD,193.31,USD,170.95,USD,171.22,USD,171.22,USD,171.22,USD,171.22,USD,171.495,USD,171.495,USD,171.495,USD,184.025,USD,1.683034e+06,USD,170.798,170.805000,170.83900,170.928046,171.115794,170.863100,170.610406,171.229721,170.965845,170.701969,US02079K1079,0.738561,2024-10-29,2025-02-04,Buy,209.563636,USD,Sell
1,GOOGL,Alphabet Inc.,Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500 Communication Services, S&P 100, ISE CTA Cloud Computing, Nasdaq US Large Cap Growth, Russell 3000, NASDAQ-100 Technology Sector, Russell 1000, NASDAQ Computer, S&P 500 ESG, NASDAQ CB Insights Metaverse US Index",2.078499e+12,USD,169.12,USD,169.13,USD,0.27,USD,0.159650,0.005913,5.828977e+09,20486647,53621983.0,36654,30129536.3,2.800292e+07,2.651506e+07,1.336675,2.667327,2.278010,168.840,22.434171,0.697706,0.669622,1.037961,0.979892,1.033457,4.851180,6.759130,6.215892,3.079556,1.256359,0.547016,0.397163,0.421984,0.864261,182.490,USD,182.490,USD,191.75,USD,191.75,USD,191.75,USD,169.40,USD,169.60,USD,169.60,USD,169.60,USD,169.60,USD,169.820,USD,169.820,USD,169.820,USD,182.490,USD,1.683034e+06,USD,169.243,169.238750,169.24598,169.319215,169.510918,169.271505,169.032092,169.588697,169.351495,169.114293,US02079K3059,0.669622,2024-10-29,2025-02-04,Strong buy,209.678571,USD,Sell
2,META,"Meta Platforms, Inc.",Internet s

# LEVEL #1
| PRIMARY SCREENING |
|-|
| LEVEL #1 |

In [27]:
# CREATE CATEGORIES FOR MARKET CAP
def categorize_market_cap(df):
    """Categorize stocks based on market capitalization."""
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df

# execute categorization
category_setup_df = categorize_market_cap(trading_view_df).copy()

# drop Undefined marketCapType
category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume'
]

# Apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

# CRITERIA CONFIGURATION FOR EACH MARKET CAP CATEGORY
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    # "Midlers" in TradingView
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

# FILTER STOCKS BASED ON CONFIGURATION CRITERIA - STAGING
def filter_stocks(df, config):
    """Filter stocks based on configuration criteria."""
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

# SCREEN STOCKS BY CATEGORY
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# EXECUTE KEY SCREENING FUNCTION AND CREATE DATAFRAME
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# rendered column list and ordering
cols_list = [
    'Symbol', 
    'Description', 
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'marketCapType',
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Pre-market Change %', 'Price'],
    ascending=[False, False]).reset_index(drop=True)


print(f"{smash_df.shape[0]} stockes found in the screener.")

43 stockes found in the screener.


In [28]:
# # display(smash_df.head(4))
# print(len(smash_df))
# print(smash_df)

| TECHNICAL ANALYSIS |
|-|
| LEVEL #2 |

RVOL retriever ????

# Share and / or some of these to setup KEY LEVELS

In [29]:
import yfinance as yf
from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator

In [30]:
print(screen_date)

2024-11-27


fix all this shit. yahoo will give you close. 
you get pre-market from tradingview (0615?)

In [31]:
def technical_screen(df, test_date, lookback_period=100):
    """
    Calculate technical indicators using TA library and apply filters for a specific date.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing stock symbols
    test_date (str): Date to test in 'YYYY-MM-DD' format
    lookback_period (int): Number of days to look back for calculating indicators
    
    Returns:
    pd.DataFrame: DataFrame with technical indicators and filtered results
    """
    # Calculate start date for historical data
    start_date = (datetime.strptime(test_date, '%Y-%m-%d') - 
                 timedelta(days=lookback_period)).strftime('%Y-%m-%d')
    end_date = (datetime.strptime(test_date, '%Y-%m-%d') + 
                timedelta(days=1)).strftime('%Y-%m-%d')
    
    symbols = df['Symbol'].tolist()
    technical_data = []
    
    for symbol in symbols:
        try:
            # Get historical data
            ticker = yf.Ticker(symbol)
            hist = ticker.history(start=start_date, end=end_date, interval='1d')
            
            if not hist.empty:
                # Calculate indicators using TA library
                close_prices = hist['Close']
                
                # RSI
                rsi = RSIIndicator(close=close_prices, window=14)
                hist['RSI'] = rsi.rsi()
                
                # Moving Averages
                sma5 = SMAIndicator(close=close_prices, window=5)
                sma20 = SMAIndicator(close=close_prices, window=20)
                sma50 = SMAIndicator(close=close_prices, window=50)
                # sma100 = SMAIndicator(close=close_prices, window=100)
                # sma200 = SMAIndicator(close=close_prices, window=200)
                # macd = MACD(close=close_prices)

                hist['MA5'] = sma5.sma_indicator()
                hist['MA20'] = sma20.sma_indicator()
                hist['MA50'] = sma50.sma_indicator()
                # hist['MA100'] = sma100.sma_indicator()
                # hist['MA200'] = sma200.sma_indicator()
                # hist['MACD'] = macd.macd()
                
                # Get most recent values
                current_price = hist['Close'].iloc[-1]
                current_rsi = hist['RSI'].iloc[-1]
                current_ma5 = hist['MA5'].iloc[-1]
                current_ma20 = hist['MA20'].iloc[-1]
                current_ma50 = hist['MA50'].iloc[-1]
                # current_ma100 = hist['MA100'].iloc[-1]
                # current_ma200 = hist['MA200'].iloc[-1]
                # current_macd = hist['MACD'].iloc[-1]                
                
                technical_data.append({
                    'Symbol': symbol,
                    'yahooPrice': current_price,
                    'RSI': current_rsi,
                    'MA5': current_ma5,
                    'MA20': current_ma20,
                    'MA50': current_ma50,
                    # 'MA100': current_ma100,
                    # 'MA200': current_ma200,
                    # 'MACD': current_macd,
                    'Volume': hist['Volume'].iloc[-1],
                    # add curret volume ???? data source ????
                    # 'RVOL8': hist['Volume'].iloc[-1] / hist['Volume'].rolling(8).mean().iloc[-1],
                    'Date': hist.index[-1]
                })
                
        except Exception as e:
            print(f"Error processing {symbol}: {str(e)}")
            continue
    
    result_df = pd.DataFrame(technical_data)
    return result_df


In [32]:
explode_df = technical_screen(smash_df, screen_date)

explode_df

$CWEN.A: possibly delisted; no timezone found


,Symbol,yahooPrice,RSI,MA5,MA20,MA50,Volume,Date
0,URBN,40.139999,63.733221,38.770000,37.786000,37.240800,4299200,2024-11-26 00:00:00-05:00
1,MNSO,17.400000,47.257778,17.134000,18.286000,17.724800,1137900,2024-11-26 00:00:00-05:00
2,JAZZ,122.790001,56.783413,123.627998,119.486000,114.169400,560300,2024-11-26 00:00:00-05:00
3,VNO,43.990002,60.841768,42.375999,42.485500,41.310000,1434500,2024-11-26 00:00:00-05:00
4,SKM,23.250000,61.418796,22.824000,22.561000,23.026800,244200,2024-11-26 00:00:00-05:00
5,KT,17.410000,72.353010,16.662000,15.817500,15.564400,3417300,2024-11-26 00:00:00-05:00
6,VSH,18.190001,62.141915,16.394000,16.679000,17.566800,7977400,2024-11-26 00:00:00-05:00
7,WERN,40.849998,61.247914,39.860000,39.557500,38.129927,666800,2024-11-26 00:00:00-05:00
8,EXTR,16.940001,64.608842,16.110000,15.981500,15.353400,1138400,2024-11-26 00:00:00-05:00
9,CARG,36.459999,69.186873,35.760000,34.478000,32.133000,805900,2024-11-26 00:00:00-05:00


In [33]:
# [print(smash_df.columns)]

In [34]:
# merge join the two dataframes
explode_df = pd.merge(smash_df[['Symbol', 
                                'Description', 
                                'Exchange',
                                'Index',
                                'Sector',
                                'Float shares outstanding',
                                'marketCapType', 
                                'Pre-market Change %', 
                                'Pre-market Gap %', 
                                'Market capitalization', 
                                'Price', 'Pre-market Open', 
                                'Relative Volume 5 minutes', 
                                'Industry',
                                'Analyst Rating',
                                'Technical Rating 5 minutes']], explode_df, on='Symbol', how='inner')


print(len(explode_df))
print(len(explode_df.columns))
display(explode_df.head(2))
# print(explode_df)

42
24


,Symbol,Description,Exchange,Index,Sector,Float shares outstanding,marketCapType,marketCapType,Pre-market Change %,Pre-market Gap %,Market capitalization,Price,Pre-market Open,Relative Volume 5 minutes,Industry,Analyst Rating,Technical Rating 5 minutes,yahooPrice,RSI,MA5,MA20,MA50,Volume,Date
0,URBN,"Urban Outfitters, Inc.",NASDAQ,"NASDAQ Composite, Russell 2000, Russell 3000, NASDAQ Industrials, Mini-Russell 2000",Retail trade,5.794666e+07,Mid caps,Mid caps,13.976084,8.246139,3.703328e+09,40.14,43.45,6.739328,Apparel/Footwear retail,Neutral,Buy,40.139999,63.733221,38.770,37.786,37.2408,4299200,2024-11-26 00:00:00-05:00
1,MNSO,MINISO Group Holding Limited,NYSE,NASDAQ Golden Dragon China,Retail trade,3.072968e+08,Mid caps,Mid caps,8.275862,7.471264,5.502260e+09,17.40,18.70,9.155444,Specialty stores,Buy,Strong buy,17.400000,47.257778,17.134,18.286,17.7248,1137900,2024-11-26 00:00:00-05:00


operations-START

# LEVEL [-1] or [-2] (END (or near the END))
| FINAL MOMENTUM SCREENER (& filter conditions) |
|-|
| LEVEL #[-1] or 2nd to last |

In [35]:
def final_momentum_filter(df):
    """
    Final filter for high-probability morning momentum trades
    """
    return df[
        # Strong pre-market movement but not overextended
        (
            (df['Pre-market Change %'] > 3.0) |
            ((df['Pre-market Change %'] > 1.5) & (df['Relative Volume 5 minutes'] > 5.0))
        ) &
        
        # Volume validation
        (df['Relative Volume 5 minutes'] > 2.0) &
        
        # Industry/sector grouping
        (df.groupby('Industry')['Pre-market Change %'].transform('count') > 1)
            ].sort_values('Pre-market Change %', ascending=False)
    # ].sort_values('Pre-market Change %', ascending=False).head(3)

In [36]:
pmgus_sl_df = final_momentum_filter(explode_df)

In [37]:
print(len(pmgus_sl_df))
print(pmgus_sl_df.columns)
# print(pmgus_sl_df)

pmgus_sl_df.head(4)

7
Index(['Symbol', 'Description', 'Exchange', 'Index', 'Sector',
       'Float shares outstanding', 'marketCapType', 'marketCapType',
       'Pre-market Change %', 'Pre-market Gap %', 'Market capitalization',
       'Price', 'Pre-market Open', 'Relative Volume 5 minutes', 'Industry',
       'Analyst Rating', 'Technical Rating 5 minutes', 'yahooPrice', 'RSI',
       'MA5', 'MA20', 'MA50', 'Volume', 'Date'],
      dtype='object')


,Symbol,Description,Exchange,Index,Sector,Float shares outstanding,marketCapType,marketCapType,Pre-market Change %,Pre-market Gap %,Market capitalization,Price,Pre-market Open,Relative Volume 5 minutes,Industry,Analyst Rating,Technical Rating 5 minutes,yahooPrice,RSI,MA5,MA20,MA50,Volume,Date
1,MNSO,MINISO Group Holding Limited,NYSE,NASDAQ Golden Dragon China,Retail trade,3.072968e+08,Mid caps,Mid caps,8.275862,7.471264,5.502260e+09,17.40,18.70,9.155444,Specialty stores,Buy,Strong buy,17.400000,47.257778,17.134000,18.2860,17.7248,1137900,2024-11-26 00:00:00-05:00
2,JAZZ,Jazz Pharmaceuticals plc,NASDAQ,"NASDAQ Composite, S&P MidCap 400, Russell 3000, Russell 1000, NASDAQ Biotechnology",Health technology,5.820575e+07,Mid caps,Mid caps,7.598339,7.598339,7.423220e+09,122.79,132.12,6.156763,Pharmaceuticals: major,Buy,Buy,122.790001,56.783413,123.627998,119.4860,114.1694,560300,2024-11-26 00:00:00-05:00
3,VNO,Vornado Realty Trust,NYSE,"S&P MidCap 400, Nasdaq US Small Cap Growth, Russell 3000, Russell 1000",Finance,1.756476e+08,Mid caps,Mid caps,4.569220,0.932030,8.386653e+09,43.99,44.40,4.232255,Real estate investment trusts,Neutral,Buy,43.990002,60.841768,42.375999,42.4855,41.3100,1434500,2024-11-26 00:00:00-05:00
9,CARG,"CarGurus, Inc.",NASDAQ,"NASDAQ Composite, Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, NASDAQ Computer, Mini-Russell 2000",Technology services,8.365660e+07,Mid caps,Mid caps,2.331322,0.356555,3.781195e+09,36.46,36.59,11.174099,Internet software/Services,Buy,Strong buy,36.459999,69.186873,35.760000,34.4780,32.1330,805900,2024-11-26 00:00:00-05:00


In [38]:
# export to csv
short_list_fp_fn = f'/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/pmgus_short_list_dataframe_output_{screen_date}.csv'
pmgus_sl_df.to_csv(short_list_fp_fn, index=False)

# NEWS

NEWS resources (alt) Re:IT news data source;
NewsAPI.org Documentation: https://newsapi.org/docs
GNews API Documentation: https://gnews.io/docs/v4
Natural Language Toolkit (NLTK): https://www.nltk.org/ (supports multiple languages)
spaCy (NLP Library): https://spacy.io/usage/models (has models for Italian)



In [39]:
print(len(pmgus_sl_df))
print(pmgus_sl_df.columns)

7
Index(['Symbol', 'Description', 'Exchange', 'Index', 'Sector',
       'Float shares outstanding', 'marketCapType', 'marketCapType',
       'Pre-market Change %', 'Pre-market Gap %', 'Market capitalization',
       'Price', 'Pre-market Open', 'Relative Volume 5 minutes', 'Industry',
       'Analyst Rating', 'Technical Rating 5 minutes', 'yahooPrice', 'RSI',
       'MA5', 'MA20', 'MA50', 'Volume', 'Date'],
      dtype='object')


## NEWS - yahoo finance

In [40]:
# Retrieve the first Symbol from the pmgus_sl_df DataFrame
the_stock_str = pmgus_sl_df['Symbol'].iloc[0]
# verify string dtype
the_stock_str = str(the_stock_str)

# print the first Symbol as str
print(the_stock_str)

MNSO


In [41]:
# get yahoo finance NEWS data
def get_stock_news_df(ticker_symbol, hours_lookback=24):
    """
    Creates a DataFrame of news articles for a given ticker
    
    Parameters:
    -----------
    ticker_symbol : str
        The stock symbol to get news for
    hours_lookback : int
        Number of hours to look back for news
        
    Returns:
    --------
    pandas.DataFrame with columns:
        - Symbol
        - Headline
        - Publisher
        - Time
        - Link
    """
    # Get the ticker object
    stock = yf.Ticker(ticker_symbol)
    
    # Get all news
    news_data = stock.get_news()
    
    # Create lists to store our data
    articles = []
    
    # Process each news article
    for article in news_data:
        articles.append({
            'Symbol': ticker_symbol,
            'Headline': article['title'],
            'Publisher': article['publisher'],
            'Time': datetime.fromtimestamp(article['providerPublishTime']).strftime('%Y-%m-%d %H:%M:%S'),
            'Link': article['link']
        })
    
    # Create DataFrame
    yf_news_df = pd.DataFrame(articles)
    
    # Sort by time, most recent first
    yf_news_df = yf_news_df.sort_values('Time', ascending=False)
    
    return yf_news_df


# EXECUTE
yf_news_df = get_stock_news_df(the_stock_str)
display(yf_news_df.head(5))

,Symbol,Headline,Publisher,Time,Link
0,MNSO,"MINISO Announces Extraordinary General Meeting on January 17, 2025",PR Newswire,2024-11-22 01:30:00,https://finance.yahoo.com/news/miniso-announces-extraordinary-general-meeting-093000898.html
1,MNSO,Here's Why MINISO Group Holding (NYSE:MNSO) Has Caught The Eye Of Investors,Simply Wall St.,2024-11-17 06:10:32,https://finance.yahoo.com/news/heres-why-miniso-group-holding-141032038.html
2,MNSO,Magical Wonders Await: MINISO Launches 800+ Harry Potter Inspired Products Globally,PR Newswire,2024-11-15 02:30:00,https://finance.yahoo.com/news/magical-wonders-await-miniso-launches-103000515.html
3,MNSO,"MINISO Group Will Report September Quarter 2024 Financial Results on November 29, 2024",PR Newswire,2024-11-15 01:00:00,https://finance.yahoo.com/news/miniso-group-report-september-quarter-090000045.html
4,MNSO,Brokers Suggest Investing in MINISO Group Holding Limited (MNSO): Read This Before Placing a Bet,Zacks,2024-11-14 06:30:10,https://finance.yahoo.com/news/brokers-suggest-investing-miniso-group-143010963.html


more news add to the existing df
-wsj api?
-nyt api
-tradingview?
-marketwatch veteran discount and sign up
-real time streaming?
-markov chains, stochastic vars
-auto regression, + ar vectors
-stocktwits?

### NEWS - NewsAPI

IMPORTANT! - note the the free version has 24hr delay on news 

In [42]:
class NewsAPIClient:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv('NewsAPI_KEY')
        self.base_url = 'https://newsapi.org/v2'
        self.headers = {'X-Api-Key': self.api_key}
    
    def search_company_news(self, company, days_back=7, language='en', sort_by='publishedAt'):
        """
        Search for company-related news articles
        
        Args:
            company (str): Company name or ticker
            days_back (int): Number of days to look back
            language (str): Article language (e.g., 'en' for English)
            sort_by (str): Sort method ('relevancy', 'popularity', 'publishedAt')
        """
        from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
        
        params = {
            'q': company,
            'from': from_date,
            'language': language,
            'sortBy': sort_by,
        }
        
        response = requests.get(
            f'{self.base_url}/everything',
            headers=self.headers,
            params=params
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"API request failed with status {response.status_code}: {response.text}")

# EXECUTE news api data retrieval
try:
    news_client = NewsAPIClient()
    
    # Verify API key is loaded
    if not news_client.api_key:
        raise ValueError("API key not found in .env file. Please check your NewsAPI_KEY variable.")
    
    # # Define the stock string before using it
    # the_stock_str = "TSLA"  # or whatever stock you want to search for
    
    # Test with a sample request
    the_stock_news = news_client.search_company_news(company=the_stock_str, days_back=1)
    
    print(f"Total articles found: {len(the_stock_news['articles'])}")
    # Print article to output
    if the_stock_news.get('articles'):
        # all articles available
        for article in the_stock_news['articles'][:10]:
            # print the length or number of articles returned
            print("\nArticle Details:")
            print(f"Title: {article['title']}")
            print(f"Source: {article['source']['name']}")
            # Fix the datetime parsing - publishedAt comes as ISO format string
            published_date = datetime.fromisoformat(article['publishedAt'].replace('Z', '+00:00'))
            print(f"Published At: {published_date.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"Link: {article['url']}")

    else:
        print("No articles found in the response")

except Exception as e:
    print(f"Error: {str(e)}")

Total articles found: 0
No articles found in the response


NEWS - Italian NEWS

NEWS - other ????

LEVELS

In [43]:
""" 
NOTES: What key levels should i use? start with a lot, then refine later maybe
try to sort by importance, or weight, or confluence.
  
list of key levels brainstorm. try to order this by weight (confluence);
- pre-market high
- pre-market low
- pre-market open
- yesterday's high
- yesterday's low
- yesterday's close
- yesterday's open
- 52 week high
- 52 week low
- MA50
- MA200
- MA20
- MA5
- VWAP
- fibonacci retracement levels

OTHER;
pivot points
RSI
MACD
POC
actual price
pre-market close

"""

" \nNOTES: What key levels should i use? start with a lot, then refine later maybe\ntry to sort by importance, or weight, or confluence.\n  \nlist of key levels brainstorm. try to order this by weight (confluence);\n- pre-market high\n- pre-market low\n- pre-market open\n- yesterday's high\n- yesterday's low\n- yesterday's close\n- yesterday's open\n- 52 week high\n- 52 week low\n- MA50\n- MA200\n- MA20\n- MA5\n- VWAP\n- fibonacci retracement levels\n\nOTHER;\npivot points\nRSI\nMACD\nPOC\nactual price\npre-market close\n\n"

operations-END

# DATA ENRICHMENT
| ENRICH DATA with other data sources |
|-|
| Enrich and augment the dataframe |

# Perfomance Testing
| Perfomance Testing (after market close for DAY)|
|-|
| test the performance of the earlier selected target PMGU stock |

In [44]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import pytz

In [45]:
# def plot_daily_price(symbol, date_str):
#     """
#     Plot stock price action for a single trading day with EST time
#     """
#     # Get data
#     start_date = pd.to_datetime(date_str)
#     end_date = start_date + timedelta(days=1)
    
#     # Download 1-minute data
#     ticker = yf.Ticker(symbol)
#     df = ticker.history(start=start_date, end=end_date, interval='1m')
    
#     # Convert index to Eastern Time
#     df.index = df.index.tz_convert('US/Eastern')
    
#     # Filter for market hours (9:30 AM - 4:00 PM ET)
#     market_open = pd.Timestamp(date_str + ' 09:30:00', tz='US/Eastern')
#     market_close = pd.Timestamp(date_str + ' 16:00:00', tz='US/Eastern')
#     df = df.between_time('9:30', '16:00')
    
#     # Create plot with dark theme
#     plt.style.use('dark_background')
#     fig, ax = plt.subplots(figsize=(15, 8))
    
#     # Plot price
#     ax.plot(df.index, df['Close'], color='white', linewidth=1)
    
#     # Add VWAP
#     df['Cumulative_PV'] = (df['Close'] * df['Volume']).cumsum()
#     df['Cumulative_Volume'] = df['Volume'].cumsum()
#     df['VWAP'] = df['Cumulative_PV'] / df['Cumulative_Volume']
#     ax.plot(df.index, df['VWAP'], color='purple', alpha=0.7, linestyle='--', label='VWAP')
    
#     # Format x-axis for EST time
#     ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=pytz.timezone('US/Eastern')))
#     ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
#     # set every 5 mins ticks for plot
#     ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=15))
#     ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M', tz=pytz.timezone('US/Eastern')))
#     plt.xticks(rotation=45)
    
#     # Add title and labels
#     plt.title(f'${symbol} Price Action on {date_str}', color='white', pad=20)
#     plt.ylabel('Price ($)', color='white')
    
#     # Add grid
#     ax.grid(True, alpha=0.2)
    
#     # Add entry price reference line
#     entry_price = 22.63
#     ax.axhline(y=entry_price, color='yellow', linestyle='--', alpha=0.5, label='Entry Price')
    
#     # Add legend
#     plt.legend()
    
#     # Adjust layout and margins
#     plt.tight_layout()
    
#     plt.show()


# #### EXECUTE for TARGET ticker symbol (1) ####
# plot_daily_price(the_stock_str, screen_date)

In [46]:
# addding KEY LEVELS
# EPs, SLs, RLs, TPs
# pre-market high, low, open, close
# yesterday, high, low, open, close
# fib levels
# auto regression
# vector auto regression
# MAs
# VWAP
# Volume
# RSI
# MACD

In [47]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
import numpy as np
from datetime import datetime, timedelta

def plot_daily_price_levels(symbol, target_date):
    """
    Enhanced intraday price action plot with properly scaled price levels
    """
    # Convert target_date to datetime and calculate previous day
    target_date = pd.to_datetime(target_date)
    prev_day = target_date - pd.DateOffset(days=1)
    
    # Get previous day's data for levels
    ticker = yf.Ticker(symbol)
    prev_data = ticker.history(start=prev_day, end=target_date, interval='1d')
    
    # Get target day's data with 15-min intervals
    today_data = ticker.history(start=target_date, end=target_date + pd.DateOffset(days=1), interval='1m')
    
    if today_data.empty:
        print(f"No data available for {symbol} on {target_date.date()}")
        return
    
    # Timezone handling
    if today_data.index.tz is not None:
        today_data.index = today_data.index.tz_convert('US/Eastern')
    else:
        today_data.index = today_data.index.tz_localize('UTC').tz_convert('US/Eastern')
    
    # Filter for market hours only (9:30 AM - 4:00 PM ET)
    today_data = today_data.between_time('9:30', '16:00')
    
    # Create plot
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(14, 9))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0)
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])
    
    # Plot price action
    ax1.plot(today_data.index, today_data['Close'], 
             color='white', linewidth=1.5, label='Price')
    
    # Calculate current day levels
    today_open = today_data['Open'].iloc[0]
    today_high = today_data['High'].max()
    today_low = today_data['Low'].min()
    today_close = today_data['Close'].iloc[-1]
    
    # Calculate all price levels for scaling
    all_price_levels = []
    if not prev_data.empty:
        prev_day_high = prev_data['High'].iloc[-1]
        prev_day_low = prev_data['Low'].iloc[-1]
        prev_day_close = prev_data['Close'].iloc[-1]
        
        all_price_levels.extend([
            prev_day_high, prev_day_low, prev_day_close,
            today_high, today_low, today_open, today_close
        ])
        
        # Calculate proper y-axis limits
        min_price = min(all_price_levels)
        max_price = max(all_price_levels)
        price_range = max_price - min_price
        
        # Add padding (10% of the total range)
        padding = price_range * 0.1
        y_min = min_price - padding
        y_max = max_price + padding
        
        ax1.set_ylim(y_min, y_max)
        
        # Previous day levels (dashed lines)
        ax1.axhline(y=prev_day_high, color='red', linestyle='--', alpha=0.4)
        ax1.axhline(y=prev_day_low, color='green', linestyle='--', alpha=0.4)
        ax1.axhline(y=prev_day_close, color='yellow', linestyle='--', alpha=0.4)
        
        # Current day levels (solid lines)
        ax1.axhline(y=today_high, color='red', linestyle='-', alpha=0.3)
        ax1.axhline(y=today_low, color='green', linestyle='-', alpha=0.3)
        
        # Add labels with adjusted positioning
        label_offset = price_range * 0.02
        
        # Previous day levels on right side
        ax1.text(today_data.index[-1], prev_day_high + label_offset, f' PDH: ${prev_day_high:.2f}', 
                 color='red', alpha=0.8, va='bottom', ha='left')
        ax1.text(today_data.index[-1], prev_day_low - label_offset, f' PDL: ${prev_day_low:.2f}', 
                 color='green', alpha=0.8, va='top', ha='left')
        ax1.text(today_data.index[-1], prev_day_close, f' PDC: ${prev_day_close:.2f}', 
                 color='yellow', alpha=0.8, va='center', ha='left')
        
        # Current day levels on left side
        ax1.text(today_data.index[0], today_high + label_offset, f' H: ${today_high:.2f} ', 
                 color='red', alpha=0.8, va='bottom', ha='right')
        ax1.text(today_data.index[0], today_low - label_offset, f' L: ${today_low:.2f} ', 
                 color='green', alpha=0.8, va='top', ha='right')
        ax1.text(today_data.index[0], today_open, f' O: ${today_open:.2f} ', 
                 color='white', alpha=0.8, va='center', ha='right')
    
    # Plot volume with enhanced colors
    colors = ['red' if close < open else '#00ff00' 
              for close, open in zip(today_data['Close'], today_data['Open'])]
    ax2.bar(today_data.index, today_data['Volume'], 
            color=colors, alpha=0.7, width=0.7/(len(today_data.index)))
    
    # Format x-axis
    for ax in [ax1, ax2]:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=pytz.timezone('US/Eastern')))
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=1, tz=pytz.timezone('US/Eastern')))
        ax.grid(True, alpha=0.15)
        ax.set_xlim(today_data.index[0] - pd.Timedelta(minutes=5), 
                    today_data.index[-1] + pd.Timedelta(minutes=5))
    
    # Remove x-axis labels from top subplot
    ax1.set_xticklabels([])
    
    # Customize appearance
    fig.suptitle(f'${symbol} Intraday Price Action (15min) - {target_date.strftime("%Y-%m-%d")}', 
                 y=0.95, fontsize=12)
    ax1.set_ylabel('Price ($)', fontsize=10)
    ax2.set_ylabel('Volume', fontsize=10)
    
    # Add last price annotation
    ax1.text(0.02, 0.98, f'Last: ${today_close:.2f}', 
             transform=ax1.transAxes, color='white', 
             fontsize=10, va='top', ha='left')
    
    plt.show()
    
    return {
        'prev_day_high': prev_day_high if not prev_data.empty else None,
        'prev_day_low': prev_day_low if not prev_data.empty else None,
        'prev_day_close': prev_day_close if not prev_data.empty else None,
        'today_open': today_open,
        'today_high': today_high,
        'today_low': today_low,
        'today_close': today_close
    }

In [48]:
# plot
# plot_daily_price_levels('AAPL', '2024-11-26')
plot_daily_price_levels(the_stock_str, screen_date)

$MNSO: possibly delisted; no price data found  (1m 2024-11-27 00:00:00 -> 2024-11-28 00:00:00)


No data available for MNSO on 2024-11-27


# In Progress Below

In [49]:
# #### Export data from fidelity
# ### LOAD fidelity trade history data
# ## clean data for PMGUS trades only to df
# # plot trade BUYs/SELLs from df on existing chart
# # think about embedded data like the EP, SPs, etc. on/in the chart
# # like informational or part of informational one pager

# #import fidelity trade history data 
# fidelity_fp_fn = f'/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/fidelity_Accounts_History_{screen_date}.csv'
# fidelity_df = pd.read_csv(fidelity_fp_fn)

# display(fidelity_df.head(1))
# # display(fidelity_df)

In [50]:
# # ensure proper data types
# fidelity_df['Symbol'] = fidelity_df['Symbol'].astype(str)
# fidelity_df['Symbol'] = fidelity_df['Symbol'].str.strip()
# fidelity_df['Symbol'] = fidelity_df['Symbol'].str.upper()
# print(target_ticker_str)

In [51]:
# # return only for symbol in the target_ticker_str
# fidelity_df = fidelity_df[fidelity_df['Symbol'] == target_ticker_str]

# print(len(fidelity_df))
# # display(fidelity_df)


In [52]:
# print(fidelity_df.columns)

In [53]:
# # look for keyword BOUGHT or SOLD in the 'Action' column
# # then add new column called 'ActionSummary' with either BOUGHT or SOLD as value
# fidelity_df['ActionSummary'] = fidelity_df['Action'].apply(lambda x: 'BOUGHT' if 'BOUGHT' in x else 'SOLD')


# print(target_ticker_str)
# # print count of BOUGHT and SOLD
# print(fidelity_df['ActionSummary'].value_counts())
# # print total Amount ($) BOUGHT (negative number), add that to the Amount ($) SOLD (positive number)
# print(f"RETURN$: ${fidelity_df['Amount ($)'].sum()}")
# display(fidelity_df.head(1))

In [54]:
# def plot_daily_price_with_trades(symbol, date_str, trades_df):
#     """
#     Plot stock price action for a single trading day with trade executions marked
#     """
#     # Debug print statements
#     print("Trade times before processing:")
#     print(trades_df[['Run Date', 'ActionSummary', 'Price ($)']].head())
    
#     # Get data with proper timezone handling
#     start_date = pd.to_datetime(date_str + ' 09:30:00').tz_localize('US/Eastern')
#     end_date = pd.to_datetime(date_str + ' 16:00:00').tz_localize('US/Eastern')
    
#     # Download 1-minute data
#     ticker = yf.Ticker(symbol)
#     df = ticker.history(start=start_date, end=end_date, interval='1m')
    
#     if df.empty:
#         print(f"No data available for {symbol} on {date_str}")
#         return
        
#     # Convert index to Eastern Time
#     df.index = df.index.tz_convert('US/Eastern')
    
#     # Create plot with dark theme
#     plt.style.use('dark_background')
#     fig, ax = plt.subplots(figsize=(15, 8))
    
#     # Plot price
#     ax.plot(df.index, df['Close'], color='white', linewidth=1, label='Price')
    
#     # Plot trades with larger markers and clear colors
#     if not trades_df.empty:
#         for idx, trade in trades_df.iterrows():
#             try:
#                 # Convert trade time to datetime and localize to Eastern
#                 trade_time = pd.to_datetime(trade['Run Date'])
#                 print(f"Processing trade {idx}:")
#                 print(f"Original time: {trade_time}")
                
#                 if not trade_time.tz:
#                     trade_time = trade_time.tz_localize('US/Eastern')
#                 print(f"Localized time: {trade_time}")
                
#                 if trade['ActionSummary'] == 'BOUGHT':
#                     ax.scatter(trade_time, trade['Price ($)'], 
#                              color='lime', marker='^', s=200, 
#                              label='Buy' if 'Buy' not in ax.get_legend_handles_labels()[1] else "")
#                     print(f"Plotted BUY at {trade_time} at price {trade['Price ($)']}")
#                 else:  # SOLD
#                     ax.scatter(trade_time, trade['Price ($)'], 
#                              color='red', marker='v', s=200, 
#                              label='Sell' if 'Sell' not in ax.get_legend_handles_labels()[1] else "")
#                     print(f"Plotted SELL at {trade_time} at price {trade['Price ($)']}")
#             except Exception as e:
#                 print(f"Error plotting trade {idx}: {e}")
    
#     # Format x-axis for EST time
#     ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
#     ax.xaxis.set_major_locator(mdates.HourLocator())
#     ax.grid(True, alpha=0.2)
#     plt.xticks(rotation=45)
    
#     # Set axis limits and print them
#     ax.set_xlim(start_date, end_date)
#     print(f"\nChart x-axis limits: {start_date} to {end_date}")
    
#     # Add title and labels
#     plt.title(f'${symbol} Price Action and Trades on {date_str}', color='white', pad=20)
#     plt.ylabel('Price ($)', color='white')
    
#     # Add legend without duplicates
#     handles, labels = ax.get_legend_handles_labels()
#     by_label = dict(zip(labels, handles))
#     plt.legend(by_label.values(), by_label.keys())
    
#     plt.tight_layout()
#     plt.show()

# # Let's also examine the trades data
# print("\nFirst few rows of trades_df:")
# print(fidelity_df[['Run Date', 'ActionSummary', 'Price ($)']].head())
# print("\nData types of trades_df columns:")
# print(fidelity_df.dtypes)

In [55]:
# def plot_daily_price_with_trades(symbol, date_str, trades_df):
#     """
#     Plot stock price action with trades matched to price levels, following trading rules
#     """
#     # Define market hours in Eastern Time
#     start_date = pd.to_datetime(date_str + ' 09:30:00').tz_localize('US/Eastern')
#     end_date = pd.to_datetime(date_str + ' 16:00:00').tz_localize('US/Eastern')
#     no_sell_before = pd.to_datetime(date_str + ' 09:45:00').tz_localize('US/Eastern')
    
#     # Download data
#     ticker = yf.Ticker(symbol)
#     df = ticker.history(start=start_date - pd.Timedelta(hours=1), 
#                        end=end_date + pd.Timedelta(hours=1), 
#                        interval='1m')
    
#     if df.empty:
#         print(f"No data available for {symbol} on {date_str}")
#         return
        
#     # Ensure timezone handling
#     if df.index.tz is not None:
#         df.index = df.index.tz_convert('US/Eastern')
#     else:
#         df.index = df.index.tz_localize('US/Eastern')
    
#     # Filter to market hours only
#     df = df[(df.index >= start_date) & (df.index <= end_date)]
    
#     # Create plot with dark theme
#     plt.style.use('dark_background')
#     fig, ax = plt.subplots(figsize=(15, 8))
    
#     # Plot price
#     ax.plot(df.index, df['Close'], color='white', linewidth=1, label='Price')
    
#     executed_trades = []
    
#     # First process all BOUGHT orders
#     buy_trades = trades_df[trades_df['ActionSummary'] == 'BOUGHT'].copy()
#     print(f"Total buy trades to process: {len(buy_trades)}")  # Debug print
    
#     for _, trade in buy_trades.iterrows():
#         trade_price = trade['Price ($)']
#         price_str = f"{trade_price:.2f}"
        
#         # Find the first occurrence of this price
#         price_matches = df[df['Close'].astype(str).str.startswith(price_str[:6])]
        
#         if not price_matches.empty:
#             # Take the earliest match for this price
#             match_time = price_matches.index[0]
            
#             # Check if we already have a trade at this exact time
#             while any(t['time'] == match_time for t in executed_trades):
#                 # If so, take the next occurrence
#                 price_matches = price_matches[price_matches.index > match_time]
#                 if price_matches.empty:
#                     break
#                 match_time = price_matches.index[0]
            
#             if not price_matches.empty:
#                 executed_trades.append({
#                     'time': match_time,
#                     'price': trade_price,
#                     'type': 'BOUGHT'
#                 })
    
#     # Sort buys chronologically
#     executed_trades.sort(key=lambda x: x['time'])
#     print(f"Executed buy trades: {len(executed_trades)}")  # Debug print
    
#     # Now process SOLD orders, but only after we have 2 buys
#     if len(executed_trades) >= 2:
#         second_buy_time = sorted([t['time'] for t in executed_trades if t['type'] == 'BOUGHT'])[1]
#         sell_trades = trades_df[trades_df['ActionSummary'] == 'SOLD']
#         print(f"Total sell trades to process: {len(sell_trades)}")  # Debug print
        
#         for _, trade in sell_trades.iterrows():
#             trade_price = trade['Price ($)']
#             price_str = f"{trade_price:.2f}"
            
#             # Find price matches after second buy and 9:45 AM
#             valid_time = max(second_buy_time, no_sell_before)
#             price_matches = df[
#                 (df['Close'].astype(str).str.startswith(price_str[:6])) &
#                 (df.index >= valid_time)
#             ]
            
#             if not price_matches.empty:
#                 match_time = price_matches.index[0]
                
#                 # Handle multiple trades at same time
#                 while any(t['time'] == match_time for t in executed_trades):
#                     price_matches = price_matches[price_matches.index > match_time]
#                     if price_matches.empty:
#                         break
#                     match_time = price_matches.index[0]
                
#                 if not price_matches.empty:
#                     executed_trades.append({
#                         'time': match_time,
#                         'price': trade_price,
#                         'type': 'SOLD'
#                     })
    
#     # Sort all trades chronologically
#     executed_trades.sort(key=lambda x: x['time'])
#     print(f"Total executed trades: {len(executed_trades)}")  # Debug print
    
#     # Plot trades
#     for trade in executed_trades:
#         if trade['type'] == 'BOUGHT':
#             ax.scatter(trade['time'], trade['price'], 
#                       color='lime', marker='^', s=200, 
#                       label='Buy' if 'Buy' not in ax.get_legend_handles_labels()[1] else "",
#                       zorder=5)
#             ax.annotate(f'Buy ${trade["price"]:.2f}', 
#                       (trade['time'], trade['price']),
#                       xytext=(5, 5), 
#                       textcoords='offset points',
#                       color='lime', fontsize=8)
#         else:
#             ax.scatter(trade['time'], trade['price'], 
#                       color='red', marker='v', s=200, 
#                       label='Sell' if 'Sell' not in ax.get_legend_handles_labels()[1] else "",
#                       zorder=5)
#             ax.annotate(f'Sell ${trade["price"]:.2f}', 
#                       (trade['time'], trade['price']),
#                       xytext=(5, -15), 
#                       textcoords='offset points',
#                       color='red', fontsize=8)
    
#     # Format time axis
#     ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=pytz.timezone('US/Eastern')))
#     ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
#     ax.grid(True, alpha=0.2)
    
#     # Ensure x-axis shows market hours
#     ax.set_xlim(start_date, end_date)
    
#     # Rotate and align the tick labels so they look better
#     plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
    
#     # Add title and labels
#     plt.title(f'${symbol} Price Action and Trades on {date_str}', color='white', pad=20)
#     plt.ylabel('Price ($)', color='white')
    
#     # Add legend
#     handles, labels = ax.get_legend_handles_labels()
#     by_label = dict(zip(labels, handles))
#     plt.legend(by_label.values(), by_label.keys())
    
#     plt.tight_layout()
#     plt.show()

In [56]:
# # Execute the function
# plot_daily_price_with_trades(target_ticker_str, screen_date, fidelity_df)

NOTES:

-MARA ($22.63)
Up 11.53% pre-market
Strong crypto-sector momentum
High relative volume (2.08x)
Price Point: Good for larger share size
Day's Targets: R1 = $23.50, R2 = $24.50

-MARA (60%): $48,000
Entry: $22.63
Share Count: 2,120 shares
Target Return (R2): $4,000 (+8.3% to $24.50)
Stop: $22.15 (-$1,017 risk)

In [57]:
#### new df called tt_perf_df
### 1D, 1m, 5m, 15m price to df
## date range for gap up use case relevance?





In [58]:
#### plot the near time but historical data (usually from TODAY)
### should be doing this in the evening i.e., 2200
## smooth lines, not candlesticks
# use scipy? or the TradingView open source 
# maybe want to use TV osource later on, i remember static limitations ?







In [59]:
#### export from fidelity
### import and clean for only PMGUS
## new df called tt_trades_history_df

In [60]:
#### plot the trade buy and sell points on the same chart







# END OF WORKING CODE
# END OF WORKING CODE
# END OF WORKING CODE
# END OF WORKING CODE

In [61]:
# ts_filt_df = technical_screen_filter(explode_df)

# print(len(ts_filt_df))
# display(ts_filt_df.head(3))
# # print(explode_df)

# DEVELOPEMENT
## testing the yahoo finance API
### retrieving MOST RECENT ticker data AND similar data BUT with a SPECIFIC DATE
#### useful for backtesting later and inflight developement testing

In [62]:
# # testing yahoo finance api
# ## most recent
# ticker = "ADMA"
# data = yf.download(ticker, period="1d")

# print(data)

In [63]:
# # testing retrieval
# ## specific date
# specific_date = "2024-11-11"

# # Download the data for the specific date
# data_sd = yf.download(ticker, start=specific_date, end=(datetime.strptime(specific_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'))

# # Print the data
# print(data_sd)

| LEVEL #3 - SCREENER |
|-|
| xxx |

# DEVELOPEMENT

old but fresh still good code use this 

In [64]:
# # printing raw to dictionary to inspect
# # Get ticker object
# ticker = yf.Ticker('ADMA')

# # Print raw info dictionary
# print("\nRaw Info Dictionary:")
# for key, value in ticker.info.items():
#     print(f"{key}: {value}")

# # Show available attributes/methods
# print("\nAvailable attributes/methods:")
# for method in dir(ticker):
#     if not method.startswith('_'):
#         print(method)

In [65]:
# # return smash_list of Symbols
# smash_list = smash_df['Symbol'].tolist()


# print(len(smash_df))
# print(len(smash_list))
# print(smash_list)


return current stock price

"quick back test"
which tickers from smash_list did well on today
1-baseline buy sell w/same time b/s
2-dynamic sell same buy

1-specific day to back test
2-move code over for specific times
3-baseline times
4-other ideas time segments
5-dynamic buy windows IF only?

In [66]:
# # start----quick back test function
# print(screen_date)

In [67]:
# from datetime import datetime

# # Use datetime.strptime() where needed

In [68]:
# ####---quick start back tests
# def quick_backtest(df, day):
#     symbols = df['Symbol'].tolist()
#     stock_data = {}

#     for symbol in symbols:
#         ticker = yf.Ticker(symbol)
#         qbt_data = ticker.history(start=day, end=(datetime.strptime(day, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'), interval='1m')

#         if not qbt_data.empty:
#             stock_data[symbol] = {
#                 'High': qbt_data['High'].values[0],
#                 'Low': qbt_data['Low'].values[0],
#                 'Open': qbt_data['Open'].values[0],
#                 'Close': qbt_data['Close'].values[0],
#                 'Volume': qbt_data['Volume'].values[0],
#                 'Date': qbt_data.index[0]
#             }
#         else:
#             qbt_data[symbol] = {
#                 'High': None,
#                 'Low': None,
#                 'Open': None,
#                 'Close': None,
#                 'Volume': None,
#                 'Date': None
#             }

#     qbt_stg_df = pd.DataFrame(qbt_data).T.reset_index().rename(columns={'index': 'Symbol'})
    
#     ####

#     return qbt_stg_df

In [69]:
# # execute
# print(screen_date)

old code

In [70]:
# # add high low close
# #### ---- Quick Stary Backtesting ---- ####
# def basic_daily_technicals(df):
#     symbols = df['Symbol'].tolist()
#     stock_data = {}

#     for symbol in symbols:
#         stock_info = yf.Ticker(symbol)
#         today_data = stock_info.history(period='1d')
        
#         if not today_data.empty:
#             stock_data[symbol] = {
#                 'High': today_data['High'].values[0],
#                 'Low': today_data['Low'].values[0],
#                 'Open': today_data['Open'].values[0],
#                 'Close': today_data['Close'].values[0],
#                 'Volume': today_data['Volume'].values[0],
#                 'Date': today_data.index[0]
#             }
#         else:
#             stock_data[symbol] = {
#                 'High': None,
#                 'Low': None,
#                 'Open': None,
#                 'Close': None,
#                 'Volume': None,
#                 'Date': None
#             }

#     # create a DataFrame from the stock data dictionary
#     stock_data_df = pd.DataFrame(stock_data).T.reset_index().rename(columns={'index': 'Symbol'})
    
#     # merge the new stock data with the existing DataFrame
#     updated_df = pd.merge(df, stock_data_df, on='Symbol', how='left')
    
#     return updated_df

In [71]:
# smash_two_df =  basic_daily_technicals(smash_df)

| LEVEL #2 - SCREENER |
|-|
| TECHNICAL INDICATORS - via yahoo finance |

*note yahoo finance data source transition
*feels like this is more of a swing screen????

In [72]:
# import yfinance as yf

# from ta.trend import MACD
# from ta.momentum import RSIIndicator
# from ta.trend import SMAIndicator
# from ta.momentum import RSIIndicator, StochasticOscillator
# from ta.trend import SMAIndicator, MACD, PSARIndicator
# from ta.volatility import BollingerBands, AverageTrueRange
# from ta.volume import OnBalanceVolumeIndicator
# # import vectorbt as vbt
# # vbt.settings.set_theme('dark')


In [73]:
# import yfinance as yf
# import pandas as pd
# from ta.momentum import RSIIndicator
# from ta.trend import SMAIndicator, MACD

need to add start date as a var for retroactive testing / backtesting

In [74]:
# def compute_technical_indicators(df):
#     tech_data = []

#     for symbol in df['Symbol'].unique():
#         data = yf.download(symbol, period="6mo", interval="1d")

#         if not data.empty:
#             close_prices = data['Close'].squeeze()  # Ensure it's a 1-dimensional series
#             rsi = RSIIndicator(close_prices, window=14).rsi().iloc[-1] # last value!
#             ma20 = SMAIndicator(close_prices, window=20).sma_indicator().iloc[-1] # last value!
#             ma50 = SMAIndicator(close_prices, window=50).sma_indicator().iloc[-1] # last value!
#             macd = MACD(close_prices).macd_diff().iloc[-1] # last value!

#             tech_data.append({
#                 'Symbol': symbol,
#                 'RSI': rsi,
#                 'MA20': ma20,
#                 'MA50': ma50,
#                 'MACD': macd,
#                 'Yahoo Price': close_prices.iloc[-1], # last value! (price from Yahoo!)
#             })

#     tech_df = pd.DataFrame(tech_data)
#     return tech_df

note the last value because,
technically for back testing you want to pay attention here.
real time go time i think we want it most recent. meaning yesterday
if we are making decisions and in pre-market trading hours based on the Close from yesterday
yesterday = last Close $

In [75]:
# explode_df = compute_technical_indicators(smash_df)

# print(explode_df.shape)
# print(explode_df.head())

In [76]:
# print(explode_df.shape)
# print(explode_df.head())

# print(smash_df.shape)
# print(smash_df.iloc[:5,:3])

In [77]:
# def screen_stocks_by_technical_indicators(df):
#     # filter based on RSI, MA, and MACD
#     df = df[(df['RSI'] >= 0) & (df['RSI'] < 70)]  # RSI filter condition
#     df = df[df['Yahoo Price'] > df['MA20']]
#     df = df[df['Yahoo Price'] > df['MA50']]
#     df = df[df['MACD'] > 0]
#     """ 
#     # should i add more to this?
#     # level 3 will start to get more advanced filering
#     # dont want to overfilter to soon also and miss something
#     # patience
#     """
#     return df

In [78]:
# explode_df = screen_stocks_by_technical_indicators(explode_df)

# print(len(explode_df))
# print(explode_df.head())

In [79]:
# # return explode_list of Symbols
# explode_list = explode_df['Symbol'].tolist()


# print(len(smash_df))
# print()

# print(len(explode_df))
# print(len(explode_list))
# print(explode_list)

In [80]:
# print(explode_df)

In [81]:
# print(screen_date)

In [82]:
# # sory descending by RSI
# explode_df = explode_df.sort_values(by='RSI', ascending=True)

# print(explode_df)

| LEVEL #3 - SCREENER |
|-|
| xxx |

i think this is the place for those small tactical screeners
maybe even think about deploying dynamically for instance one above this ta technical screener filter
different days, different vols, and volatilty

see index specific
see vix
companies in same industry, same sector, same location? 

| LEVEL #4 - SCREENER |
|-|
| xxx |

could think about doing additional available (but separate now) on the subset, additional technical indicators from ta library

In [83]:
# from ta.momentum import RSIIndicator, StochasticOscillator
# from ta.trend import SMAIndicator, MACD, CCIIndicator
# from ta.volatility import BollingerBands
# from ta.volume import OnBalanceVolumeIndicator
# from ta.volatility import AverageTrueRange

In [84]:
# def compute_additional_indicators(df):
#     additional_data = []

#     for symbol in df['Symbol'].unique():
#         data = yf.download(symbol, period="6mo", interval="1d")

#         if not data.empty:
#             close_prices = data['Close'].squeeze()  # Ensure it's a 1-dimensional series
#             high_prices = data['High'].squeeze()
#             low_prices = data['Low'].squeeze()
#             volume = data['Volume'].squeeze()

#             # Compute additional indicators
#             bb = BollingerBands(close_prices)
#             atr = AverageTrueRange(high=high_prices, low=low_prices, close=close_prices)
#             stoch = StochasticOscillator(close=close_prices, high=high_prices, low=low_prices)
#             obv = OnBalanceVolumeIndicator(close=close_prices, volume=volume)
#             cci = CCIIndicator(high=high_prices, low=low_prices, close=close_prices)

#             additional_data.append({
#                 'Symbol': symbol,
#                 'Bollinger High': bb.bollinger_hband().iloc[-1],
#                 'Bollinger Low': bb.bollinger_lband().iloc[-1],
#                 'ATR': atr.average_true_range().iloc[-1],
#                 'Stochastic Oscillator': stoch.stoch().iloc[-1],
#                 'OBV': obv.on_balance_volume().iloc[-1],
#                 'CCI': cci.cci().iloc[-1],
#             })

#     additional_df = pd.DataFrame(additional_data)
#     return additional_df

In [85]:
# double_explode_df = compute_additional_indicators(explode_df)

# print(len(explode_df))
# print(len(double_explode_df))

In [86]:
# double_explode_df

In [87]:
# def filter_additional_indicators(df):
#     # Example filter criteria
#     filtered_df = df[
#         (df['Stochastic Oscillator'] < 20) &  # Stochastic Oscillator indicates oversold
#         (df['CCI'] < -100) &  # CCI indicates oversold
#         (df['ATR'] > 1)  # ATR indicates high volatility
#     ]
#     return filtered_df

In [88]:
# filtered_double_explode_df = filter_additional_indicators(double_explode_df)


# print(len(explode_df))
# print(len(filtered_double_explode_df))

| LEVEL #5,#6 - SCREENER |
|-|
| (mini) tight screeners - (toggle on and off w/comments?) |

return index data for relevant?
good time to see if that chart works opensource similar to tradingview

could also plot historical data for 53 stocks who cares. check it out on same chart?
watch a youtube
restart as new feature 

what about news
what about europe /asia price action
market and industry?

| Notes |
|-|
| END of WORKING CODE |

In [89]:
# # read data file from TradingView to pandas df
# """ 
# Market : US
# Exchange : NASDAQ, NYSE
# Pre-market Chg > $0.00 USD

# Stocks = ~1100
# """
# base_file_path = '/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/'
# file_name =  'tv_screen_gap-up_2024-10-29.csv'
# pmgus_df = pd.read_csv(base_file_path + file_name)


# print(len(pmgus_df))
# pmgus_df.tail(2)

In [90]:
# def categorize_market_cap(df):
#     """Categorize stocks based on market capitalization."""
#     df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
#     conditions = [
#         (df['Market capitalization'] >= 200_000_000_000),  # Titans
#         (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
#         (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
#         (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
#         (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
#         (df['Market capitalization'] <= 50_000_000)  # Shrimp
#     ]
#     categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
#     df['marketCapType'] = np.select(conditions, categories, default='Undefined')
#     # drop Undefined
#     df = df[df['marketCapType'] != 'Undefined']
#     # convert to numeric the Market capitalization column
#     df['Market capitalization'] = pd.to_numeric(df['Market capitalization'])
    
#     return df

In [91]:
# # APPLY - categorize market cap
# pmgus_df = categorize_market_cap(pmgus_df).copy()
# print(len(pmgus_df))


In [92]:
# # convert necessary columns to numeric
# def convert_columns_to_numeric(df, columns):
#     """Convert specified columns to numeric types."""
#     for col in columns:
#         df[col] = pd.to_numeric(df[col], errors='coerce')
#     return df

# # list of columns to convert
# numeric_columns = [
#     'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
#     'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
#     'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
#     'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume'
# ]

In [93]:
# # APPLY - convert columns to numeric
# pmgus_df = convert_columns_to_numeric(pmgus_df, numeric_columns).copy()
# print(len(pmgus_df))

In [94]:
# # Criteria configuration for each market cap category
# criteria_config = {
#     "Titans": {
#         "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
#         "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
#         "relative_volume_threshold": 1.2,
#         "relative_volume_at_time_threshold": 0.03,
#         "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
#         "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
#         "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
#     },
#     "Large caps": {
#         "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
#         "float_shares_outstanding_threshold": 200000000,  # 200 million shares
#         "relative_volume_threshold": 1.3,  # More inclusive
#         "relative_volume_at_time_threshold": 0.04,  # More inclusive
#         "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
#         "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
#         "pre_market_volume_threshold": 50000  # Minimum pre-market volume
#     },
#     # "Midlers" in TradingView
#     "Midlers": { 
#         "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
#         "float_shares_outstanding_threshold": 50000000,  # 50 million shares
#         "relative_volume_threshold": 1.3,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.02,
#         "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
#         "pre_market_volume_threshold": 50000  # Minimum pre-market volume
#     },
#     "Small caps": {
#         "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
#         "float_shares_outstanding_threshold": 20000000,  # 20 million shares
#         "relative_volume_threshold": 1.2,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.03,
#         "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
#         "pre_market_volume_threshold": 50000  # Minimum pre-market volume
#     },
#     "Micro caps": {
#         "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
#         "float_shares_outstanding_threshold": 5000000,  # 5 million shares
#         "relative_volume_threshold": 1.1,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.04,
#         "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
#         "pre_market_volume_threshold": 50000  # Minimum pre-market volume
#     },
#     "Shrimp": {
#         "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
#         "float_shares_outstanding_threshold": 1000000,  # 1 million shares
#         "relative_volume_threshold": 1.0,
#         "relative_volume_at_time_threshold": 0.05,
#         "pre_market_gap_percentage_threshold": 0.05,
#         "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
#         "pre_market_volume_threshold": 50000  # Minimum pre-market volume
#     }
# }

In [95]:
# def filter_stocks(df, config):
#     """Filter stocks based on configuration criteria."""
#     conditions = (
#         (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
#         (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
#         (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
#         (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
#         (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
#         (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
#         (df['Volatility 1 day'] >= df['Volatility 1 week']) &
#         (df['Volatility 1 day'] >= df['Volatility 1 month']) &
#         (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
#     )
#     return df[conditions]

# def screen_stocks_by_category(df, category):
#     """Filter stocks in a category using predefined criteria."""
#     config = criteria_config.get(category, {})
#     filtered_df = filter_stocks(df, config)
#     return filtered_df


In [96]:
# for category in pmgus_df['marketCapType'].unique():
#     category_df = pmgus_df[pmgus_df['marketCapType'] == category]
#     gap_up_stage_df = screen_stocks_by_category(category_df, category)
#     pmgus_two_df = pd.concat([category_df, gap_up_stage_df], ignore_index=True)

# print(len(pmgus_two_df))


enhanced volume screening -L2

In [97]:
# market_cap_volume_thresholds = {
#     "Titans": {
#         "min_pm_volume_vs_adv": 0.001,  # 0.1% of ADV minimum in pre-market
#         "min_rel_vol_5min": 1.5,        # 50% above normal 5-min volume
#         "min_rel_vol_15min": 1.3        # 30% above normal 15-min volume
#     },
#     "Large caps": {
#         "min_pm_volume_vs_adv": 0.002,  # 0.2% of ADV
#         "min_rel_vol_5min": 1.8,
#         "min_rel_vol_15min": 1.5
#     },
#     "Midlers": {
#         "min_pm_volume_vs_adv": 0.003,  # 0.3% of ADV
#         "min_rel_vol_5min": 2.0,
#         "min_rel_vol_15min": 1.7
#     },
#     "Small caps": {
#         "min_pm_volume_vs_adv": 0.004,  # 0.4% of ADV
#         "min_rel_vol_5min": 2.5,
#         "min_rel_vol_15min": 2.0
#     },
#     "Micro caps": {
#         "min_pm_volume_vs_adv": 0.005,  # 0.5% of ADV
#         "min_rel_vol_5min": 3.0,
#         "min_rel_vol_15min": 2.5
#     },
#     "Shrimp": {
#         "min_pm_volume_vs_adv": 0.008,  # 0.8% of ADV
#         "min_rel_vol_5min": 3.5,
#         "min_rel_vol_15min": 3.0
#     }
# }

# def analyze_premarket_volume_by_cap(df, conditions):
#     """
#     Enhanced volume analysis based on market cap category with progressive thresholds
#     """
#     df['PM_Volume_Ratio'] = df['Pre-market Volume'] / df['Average Volume 10 days']
#     df['Volume_Acceleration'] = df['Relative Volume 5 minutes'] / df['Relative Volume 15 minutes']
    
#     # Apply filters based on market cap category
#     conditions = []
#     for cap_type, thresholds in market_cap_volume_thresholds.items():
#         cap_condition = (
#             (df['marketCapType'] == cap_type) &
#             (df['PM_Volume_Ratio'] >= thresholds['min_pm_volume_vs_adv']) &
#             (df['Relative Volume 5 minutes'] >= thresholds['min_rel_vol_5min']) &
#             (df['Relative Volume 15 minutes'] >= thresholds['min_rel_vol_15min'])
#         )
#         conditions.append(cap_condition)
    
#     return pd.concat([df[cond] for cond in conditions])

In [98]:
# # show all pandas row width
# pd.set_option('display.max_rows', None)
# # show all pandas column width
# pd.set_option('display.max_columns', None)

In [99]:
# pmgus_two_df = analyze_premarket_volume_by_cap(pmgus_two_df, market_cap_volume_thresholds)


# print(len(pmgus_two_df))
# display(pmgus_two_df.head(5))

In [100]:
# # save to csv
# new_pmgus_df.to_csv(base_file_path + 'new_pmgus_2024-10-29_final.csv', index=False)

In [101]:
# print(len(pmgus_two_df))
# print(len(pmgus_two_df.columns))
# print(pmgus_two_df.columns)

# final L1 FILTERS

In [102]:
# print(len(pmgus_two_df))

In [103]:
# # define no negatives
# def no_negatives(df):
#     """Remove negative values in the data frame."""
#     return df[(df['Pre-market Change %'] >= 0) & (df['Pre-market Gap %'] >= 0)]

# # apply no negatives
# pmgus_two_df = no_negatives(pmgus_two_df)
# print(len(pmgus_two_df))

In [104]:
# def technical_price_filter(df):
#     """
#     Filter stocks based on their position relative to key technical levels
#     """
#     conditions = (
#         # Price near recent highs suggesting momentum
#         (df['Price'] >= df['High 1 month'] * 0.85) |  
        
#         # Price above all major SMAs showing strength
#         (df['Price'] > df['Simple Moving Average (5) 1 minute']) &
#         (df['Price'] > df['Simple Moving Average (13) 5 minutes']) &
        
#         # Price near upper Bollinger Band suggesting strength
#         (df['Price'] >= df['Bollinger Bands (20) 5 minutes, Basis'])
#     )
#     return df[conditions]

# # apply technical price filter
# pmgus_two_df = technical_price_filter(pmgus_two_df)
# print(len(pmgus_two_df))

might need to back off a little on the volatility filter?

In [105]:
# def fundamental_filter(df):
#     """
#     Filter using analyst ratings and price targets
#     """
#     conditions = (
#         # Price well below analyst targets suggesting upside
#         (df['Target price 1 year'] > df['Price'] * 1.2) &
        
#         # Strong analyst ratings
#         (df['Analyst Rating'].isin(['Strong buy', 'Buy']))
#     )
#     return df[conditions]

# # apply fundamental filter
# pmgus_two_df = fundamental_filter(pmgus_two_df)

# # sort by change percentage descending
# pmgus_two_df.sort_values('Pre-market Change %', ascending=False, inplace=True)


# print(len(pmgus_two_df))

In [106]:
# def volatility_filter(df):
#     """
#     Filter for stocks showing increasing volatility
#     """
#     conditions = (
#         # Increasing volatility pattern
#         (df['Volatility 1 day'] > df['Volatility 1 week']) &
#         (df['Volatility 1 week'] > df['Volatility 1 month']) &
        
#         # Beta filter for more responsive stocks
#         (df['Beta 1 year'] > 1.0)
#     )
#     return df[conditions]

# # apply volatility filter
# pmgus_two_df = volatility_filter(pmgus_two_df)
# print(len(pmgus_two_df))

In [107]:
# # sort by largest change percentage
# pmgus_two_df.sort_values('Pre-market Change %', ascending=False, inplace=True)

# # look at data after initial filters
# print(len(pmgus_two_df))
# # display(pmgus_two_df)

In [108]:
# # return symbol and price and analyst rating
# pmgus_two_df[['Symbol', 'Price', 'Analyst Rating', 'marketCapType']]

# ADDITIONAL GRANULAR FILTERS (when needed?)

In [109]:
# if you want another volume one slightly more granular

In [110]:
# # Price above short-term moving averages indicating immediate strength
# price_conditions = (
#     (pmgus_two_df['Price'] > pmgus_two_df['Simple Moving Average (5) 1 minute']) &
#     (pmgus_two_df['Price'] > pmgus_two_df['Simple Moving Average (8) 1 minute'])
# )

# # Apply the conditions to filter the dataframe
# pmgus_three_df = pmgus_two_df[price_conditions]

# #3
# print(len(pmgus_three_df))

In [111]:
# # Volume additioal acceleration
# vol_addtl_accel_conditions = (
#     (pmgus_two_df['Relative Volume 1 minute'] > pmgus_two_df['Relative Volume 5 minutes']) &
#     (pmgus_two_df['Relative Volume 5 minutes'] > 1.5)  &  # Strong recent volume
#     (pmgus_two_df['Relative Volume 5 minutes'] > pmgus_two_df['Relative Volume 15 minutes']) &  # Accelerating volume
#     (pmgus_two_df['Relative Volume 15 minutes'] > pmgus_two_df['Relative Volume 30 minutes'])) # building momentum 

# # Apply the conditions to filter the dataframe
# pmgus_three_df = pmgus_two_df[vol_addtl_accel_conditions]

# #3
# print(len(pmgus_three_df))

# End of additional filters when needed

In [112]:
# open source tradingview type chart view. 
# then use historical data and plat the daily for the Symbols in the final_pmgus_df